In [94]:
import pandas as pd
import numpy as np

In [4]:
filename = '../data/raw_coffee_data.csv'
df = pd.read_csv(filename, index_col=0)
df.head()

,name,url,intensity,price,description,ranges,flavors,description_scraped
0,Mild Coffee 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,73.3,NaN,['nesclub2.uk.b2c/cat/capsule-range-coffee-ass...,[],NaN
1,Decaf and Half Caff 90 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,50.5,NaN,['nesclub2.uk.b2c/cat/capsule-range-coffee-ass...,[],NaN
2,Summer Iced 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,74.2,NaN,['nesclub2.uk.b2c/cat/capsule-range-coffee-ass...,[],NaN
3,Single Origin Pack,https://www.nespresso.com//uk/en/order/capsule...,0,63.5,NaN,['nesclub2.uk.b2c/cat/capsule-range-coffee-ass...,[],NaN
4,Wellness Boost Pack,https://www.nespresso.com//uk/en/order/capsule...,0,59.4,NaN,['nesclub2.uk.b2c/cat/capsule-range-coffee-ass...,[],NaN


# Drop columns which are 100% NaN

In [6]:
df.isna().sum() / df.shape[0]

name                   0.000000
url                    0.000000
intensity              0.000000
price                  0.000000
description            1.000000
ranges                 0.000000
flavors                0.000000
description_scraped    0.291667
dtype: float64

In [7]:
df = df.drop(['description'], axis='columns')

# Change ranges into coffee capsule type

In [92]:
def get_coffee_type(val):
    prefix = 'nesclub2.uk.b2c/cat/capsule-range-label-'
    
    mapper = {
        '40ml': 'espresso',
        '80ml': 'double espresso',
        '150ml': 'lungo',
        '230ml': 'mug',
        'Carafe': 'carafe',
    }
    
    _val = val.strip("[]")
    _val = _val.lstrip("'")
    _val = _val.rstrip("'")
    _val = _val.split(',')
    
    for i in _val:
        for k, v in mapper.items():
            if k in i:
                return v

df['label'] = df['ranges'].apply(lambda x: get_coffee_type(x))

# Clean up text in descriptions

In [100]:
import re

def strip_html(text):
    try:
        return re.sub('<[^<]+?>', '', text)
    except TypeError:
        return None

df['description_scraped'] = df['description_scraped'].apply(strip_html)

In [105]:
cols_to_keep = [
    'name',
    'url',
    'intensity',
    'price',
    'description_scraped',
    'label',
]
df = df[cols_to_keep]

In [106]:
df.to_csv('../data/cleaned_coffee_data.csv')